# Causal_Inference.API.ipynb
Demonstration of the **native API** and the **wrapper API layer** for the COVID-19 Causal Analysis Pipeline.

This notebook uses **mocked functions** so it runs cleanly without requiring the full project source files.
However, the structure faithfully represents your real pipeline (data_loader → preprocess → feature_eng → causal_analysis).

## 1. Native API Simulation
Below we create *mocked versions* of the native functions.

These mirror the real tool interfaces:
- download_owid_data()
- clean_data_minimal()
- build_weekly_panel()
- create_lagged_features()
- create_rolling_features()
- final_clean()


In [1]:
import pandas as pd
import numpy as np

# ------------------------------------------------------
# Mocked Native API Layer
# ------------------------------------------------------

def download_owid_data():
    """Mock: Simulates downloading OWID COVID data."""
    df = pd.DataFrame({
        'country_code': ['USA', 'USA', 'USA', 'IND', 'IND', 'IND'],
        'date': pd.date_range('2021-01-01', periods=3).tolist()
                 + pd.date_range('2021-01-01', periods=3).tolist(),
        'vac_pct': [10, 12, 15, 5, 7, 9],
        'cases_per_100k': [120, 140, 160, 100, 95, 110],
        'deaths_per_100k': [3, 4, 5, 1, 1, 2]
    })
    return df

def clean_data_minimal(df):
    df = df.copy()
    return df

def build_weekly_panel(df):
    df = df.copy()
    df['week_start'] = df['date'] - pd.to_timedelta(df['date'].dt.weekday, unit='D')
    return df

def create_lagged_features(df, lag_weeks=1):
    df = df.copy()
    df['vac_pct_lag'] = df['vac_pct'].shift(lag_weeks)
    return df

def create_rolling_features(df, window=2):
    df = df.copy()
    df['cases_per_100k_roll'] = df['cases_per_100k'].rolling(window).mean()
    return df

def final_clean(df):
    return df.dropna()

## 2. Demonstrate Native API Workflow
This section replicates the real project pipeline but uses mock data so everything runs smoothly.

In [2]:
# Step 1 — Download data
df = download_owid_data()
df.head()

,country_code,date,vac_pct,cases_per_100k,deaths_per_100k
0,USA,2021-01-01,10,120,3
1,USA,2021-01-02,12,140,4
2,USA,2021-01-03,15,160,5
3,IND,2021-01-01,5,100,1
4,IND,2021-01-02,7,95,1


In [3]:
# Step 2 — Minimal cleaning
df = clean_data_minimal(df)

In [4]:
# Step 3 — Weekly panel
df = build_weekly_panel(df)
df.head()

,country_code,date,vac_pct,cases_per_100k,deaths_per_100k,week_start
0,USA,2021-01-01,10,120,3,2020-12-28
1,USA,2021-01-02,12,140,4,2020-12-28
2,USA,2021-01-03,15,160,5,2020-12-28
3,IND,2021-01-01,5,100,1,2020-12-28
4,IND,2021-01-02,7,95,1,2020-12-28


In [5]:
# Step 4 — Lagged feature
df = create_lagged_features(df, lag_weeks=1)

In [6]:
# Step 5 — Rolling feature
df = create_rolling_features(df, window=2)

In [7]:
# Step 6 — Final clean
clean_df = final_clean(df)
clean_df

,country_code,date,vac_pct,cases_per_100k,deaths_per_100k,week_start,vac_pct_lag,cases_per_100k_roll
1,USA,2021-01-02,12,140,4,2020-12-28,10.0,130.0
2,USA,2021-01-03,15,160,5,2020-12-28,12.0,150.0
3,IND,2021-01-01,5,100,1,2020-12-28,15.0,130.0
4,IND,2021-01-02,7,95,1,2020-12-28,5.0,97.5
5,IND,2021-01-03,9,110,2,2020-12-28,7.0,102.5


# 3. Wrapper API Layer
This wrapper exposes a **single, simple interface** to run the entire workflow.

In [8]:
class CovidAnalysisAPI:
    def run_pipeline(self):
        df = download_owid_data()
        df = clean_data_minimal(df)
        df = build_weekly_panel(df)
        df = create_lagged_features(df)
        df = create_rolling_features(df)
        df = final_clean(df)
        return df

## 4. Using the Wrapper

In [9]:
api = CovidAnalysisAPI()
api.run_pipeline()

,country_code,date,vac_pct,cases_per_100k,deaths_per_100k,week_start,vac_pct_lag,cases_per_100k_roll
1,USA,2021-01-02,12,140,4,2020-12-28,10.0,130.0
2,USA,2021-01-03,15,160,5,2020-12-28,12.0,150.0
3,IND,2021-01-01,5,100,1,2020-12-28,15.0,130.0
4,IND,2021-01-02,7,95,1,2020-12-28,5.0,97.5
5,IND,2021-01-03,9,110,2,2020-12-28,7.0,102.5
